### Config

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from chronos_mlx import ChronosPipeline
pd.set_option('display.max_columns', None)

In [4]:
year = pd.read_parquet('/Users/tomaltenborg/Documents/Master/Master thesis/Notebooks/M3 Data/M3_yearly_processed.parquet')

## Loop and Produce Forecasts for All Yearly Series

In [5]:
pipeline = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-base",
    dtype="bfloat16",
)

unique_series = year['Series'].unique()

forecasts = {}

for series_id in unique_series:
    # Filter the dataframe for the current series
    series_data = year[year['Series'] == series_id]
    
    # Ensure the data is sorted by date
    series_data = series_data.sort_values('Date')
    
    # Get the number of observations to use for forecasting
    N = series_data['N'].iloc[0]
    NF = series_data['NF'].iloc[0]
    
    # Get the date of the last observation used for training
    last_training_date = series_data['Date'].iloc[N-NF-1]  # -1 because iloc is 0-based

    # Prepare the input data for the model
    training_data = series_data['Value'].iloc[:N-NF].values  # Exclude the last NF values
    
    # Calculate forecast dates assuming yearly data
    forecast_dates = pd.date_range(start=last_training_date, periods=NF+1, freq='YS')[1:]  # Skip the first since it's the last training date
    
    # Predict the next NF values
    try:
        model_forecasts = pipeline.predict(
            context=training_data,
            prediction_length=NF,
            num_samples=N-NF,
            temperature=1.0,
            top_k=50,
            top_p=1.0
        )#.squeeze(0).squeeze(0).numpy()  # Remove batch and sample dimensions for simplicity
    except Exception as e:
        print(f"Failed to predict for series {series_id} with error: {e}")
        continue
    
    # Quantiles for uncertainty bounds
    low, median, high = np.quantile(model_forecasts[0], [0.1, 0.5, 0.9], axis=0)
    forecasts[series_id] = {
        'Date': forecast_dates,
        'low': low,
        'median': median,
        'high': high
    }

# Convert forecasts dictionary to a DataFrame for easier manipulation
forecast_df = pd.DataFrame()
for series_id, data in forecasts.items():
    df_temp = pd.DataFrame({
        'Date': data['Date'],
        'Series_ID': series_id,
        'Low': data['low'],
        'Median': data['median'],
        'High': data['high']
    })
    forecast_df = pd.concat([forecast_df, df_temp], ignore_index=True)

# forecast_df.set_index(['Date', 'Series_ID'], inplace=True)
print('Forecasting completed for all series.')

Forecasting completed for all series.


In [6]:
forecast_df.loc[forecast_df['Series_ID'] == 1]

,Date,Series_ID,Low,Median,High
0,1989-01-01,1,5272.390744,5471.703660,5748.109309
1,1990-01-01,1,5398.371550,5847.765768,6193.742906
2,1991-01-01,1,5358.885028,6195.623217,6573.565635
3,1992-01-01,1,4759.065967,6195.623217,6904.500289
4,1993-01-01,1,4371.721997,5866.568873,7227.913701
5,1994-01-01,1,4020.103926,5828.962662,7316.288297


In [7]:
forecast_df.to_parquet('M3_Chronos_year_forecast.parquet', index=False,  compression='gzip', engine='pyarrow')